In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [ ]:
import pandas as pd

In [ ]:
from datasets import load_dataset


In [ ]:
dataset = load_dataset("keivalya/MedQuad-MedicalQnADataset")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/233 [00:00<?, ?B/s]

medDataset_processed.csv:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16407 [00:00<?, ? examples/s]

In [ ]:
print(dataset.keys())

dict_keys(['train'])


In [ ]:
# Convert to pandas DataFrame for easier manipulation
train_df = pd.DataFrame(dataset['train'])

In [ ]:
train_df.head(5)

,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."


In [ ]:
# prompt: drop qtype column

train_df = train_df.drop(columns=['qtype'])

In [ ]:
train_df.head(5)

,Question,Answer
0,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."


In [ ]:
# prompt: change Question to question and Answer to answer

train_df = train_df.rename(columns={'Question': 'question', 'Answer': 'answer'})



In [ ]:
train_df.head(5)

,question,answer
0,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."


In [ ]:
import json

def format_for_llama_finetuning(df, output_file_path='medqu.jsonl'):
    """
    Formats a DataFrame with 'question' and 'answer' columns into LLaMA 2 7B compatible JSONL.
    Structure: <s>[INST] <<SYS>>...<</SYS>>Question:...[/INST] Answer...</s>
    """
    with open(output_file_path, 'w', encoding='utf-8') as jsonl_file:
        for _, row in df.iterrows():
            # Ensure strings and strip whitespace
            question = str(row['question']).strip()
            answer = str(row['answer']).strip()

            # LLaMA 2 instruction format
            llama_formatted_text = (
                f"<s>[INST] <<SYS>>\n"
                f"Provide accurate, concise answers to medical questions.\n"
                f"<</SYS>>\n\n"
                f"Question: {question} [/INST] "
                f"Answer: {answer}</s>"
            )

            # Write to JSONL
            jsonl_file.write(json.dumps({
                "text": llama_formatted_text
            }, ensure_ascii=False) + '\n')


In [ ]:

# Usage:
format_for_llama_finetuning(train_df)  # final_df has 'question' and 'answer' columns

In [ ]:
# Display the first few lines of the JSONL file to ensure it's correct
!head -n 1 medqu.jsonl

{"text": "<s>[INST] <<SYS>>\nProvide accurate, concise answers to medical questions.\n<</SYS>>\n\nQuestion: Who is at risk for Lymphocytic Choriomeningitis (LCM)? ? [/INST] Answer: LCMV infections can occur after exposure to fresh urine, droppings, saliva, or nesting materials from infected rodents.  Transmission may also occur when these materials are directly introduced into broken skin, the nose, the eyes, or the mouth, or presumably, via the bite of an infected rodent. Person-to-person transmission has not been reported, with the exception of vertical transmission from infected mother to fetus, and rarely, through organ transplantation.</s>"}


In [ ]:
from google.colab import files
files.download('medqu.jsonl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>